## Extraktionsbaserad textsammanfattare med olika rankningsmått 

Prio: 
* Hämtning | vi väljer korpus 
* Cleaning / preprocessing 
* (Under tiden fundera på om vi skulle kunna använda maskinlärningsmetoder på något sätt)

# ALGORITHM 
### Preprocessing
* Input corpus 
* Store away stopwords 

### Rankingmått 
* Headings
* Cohesion and lexical connectedness criteria
Finns underkriterier för måttet 
* NER (nltk lib)

* Store score for each sentence 

### Assemble output 
* Reassemble according to ranking 
* Output summarization 